# ノック61 - 輸送最適化問題を解く(ここから自身の解答)

前章の輸送最適化問題を、最適化計算ライブラリによって解いていく。  
ここではpulpとortoolpyを用いる。

In [61]:
import pandas as pd
import numpy as np
from pulp import LpVariable, lpSum, value
from ortoolpy import model_min, addvars,addvals

まずは各種ファイルを読み込む

In [48]:
demand = pd.read_csv("demand.csv")
demand

# 各工場が最低限受け入れなくてはならない最低限の数

,F1,F2,F3,F4
0,28,29,31,25


In [47]:
supply = pd.read_csv("supply.csv")
supply

#各倉庫が供給できる最大の数

,W1,W2,W3
0,35,41,42


In [49]:
cost = pd.read_csv("trans_cost.csv")
cost = cost.pivot_table(index = "工場")
cost

# 各倉庫から各工場への輸送費

,F1,F2,F3,F4
工場,,,,
W1,10,10,11,27
W2,18,21,12,14
W3,15,12,14,12


上記のdemand, costの制約条件を満たしつつ、各倉庫から各工場にそれぞれどの程度輸送すればコストが最も抑えられるのかを計算する。

In [50]:
mod = model_min()

a=LpVariable('a',lowBound=0)
b=LpVariable('b',lowBound=0)
c=LpVariable('c',lowBound=0)
d=LpVariable('d',lowBound=0)
e=LpVariable('e',lowBound=0)
f=LpVariable('f',lowBound=0)
g=LpVariable('g',lowBound=0)
h=LpVariable('h',lowBound=0)
k=LpVariable('k',lowBound=0)
l=LpVariable('l',lowBound=0)
m=LpVariable('m',lowBound=0)
n=LpVariable('n',lowBound=0)

mod += cost.iloc[0,0] * a + cost.iloc[0,1] * b + cost.iloc[0,2] * c + cost.iloc[0,3] * d + cost.iloc[1,0] * e + cost.iloc[1,1] * f + cost.iloc[1,2] * g + cost.iloc[1,3] * h + cost.iloc[2,0] * k + cost.iloc[2,1] * l + cost.iloc[2,2] * m + cost.iloc[2,3] * n 

mod += a + b + c + d <= 35
mod += e + f + g + h <= 41
mod += k + l + m + n <= 42
mod += a + e + k >= 28
mod += b + f + l >= 29
mod += c + g + m >= 31
mod += d + h + n >= 25

mod.solve()

1

In [52]:
# 結果

print("総輸送コスト : " + str(value(mod.objective)))

総輸送コスト : 1296.0


In [86]:
# 輸送量をデータフレーム化

result_list = [[value(a),value(b),value(c),value(d)],[value(e),value(f),value(g),value(h)],[value(k),value(l),value(m),value(n)]]

result = pd.DataFrame(result_list, index = ["W1","W2","W3"], columns = ["F1","F2","F3","F4"])
result

,F1,F2,F3,F4
W1,28.0,7.0,0.0,0.0
W2,0.0,0.0,31.0,5.0
W3,0.0,22.0,0.0,20.0


上記のような輸送ルートと輸送量によって、総輸送コストが1296に抑えられる事が分かった。  